# **ANÁLISE DOS MICRODADOS DO ENEM 2019**






O objetivo é realizar uma análise quantitativa sobre a dinâmica dos perfis inscritos no ENEM 2019, tendo em vista as condições socioeconômicas ou quaisquer outras características que possam ser o reflexo do cenário socioeconômico nacional.

Os dados estão no Google Drive, portanto, é necessário que seja montado. É possível fazer isso acessando o último item do menu lateral da página inicial (Arquivos) e clicando em "Montar Drive" representado por um símbolo de uma pasta com a logo do Google Drive.
Outra forma é seguindo as instruções da célula abaixo.

In [1]:
## Caso o Google Drive não estiver montado ainda, retirar os comentários das linhas de código abaixo
## (aquelas que possuem somente um '#').

## ATENÇÃO! Aparecerá um link externo seguro provido automaticamente pelo Google Colab para que seja 
## gerado o código de autenticação necessário para o acesso ao Drive.

## Linhas de código a descomentar:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Bibliotecas



Bibliotecas utilizadas:

- *pandas*: para carregar os dados em um dataframe.

- *numpy*: para realizar operações numéricas

- *seaborn*: para plotar gráficos estatisticos

- *matplotlib*: para plotar gráficos



- *zipfile*: para descompactar arquivos

- *string*: para manipular strings

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from zipfile import ZipFile
import string

# Coleta e Pré-processamento de Dados

Extraindo o arquivo .csv da pasta .zip contendo microdados do ENEM 2019


In [3]:
# complete original file stored on this Drive 
zipdata_path = '/content/drive/My Drive/Colab Notebooks/Dados/microdados_enem_2019.zip'

with ZipFile(zipdata_path, 'r') as zipfile:
  zipfile.extract('DADOS/MICRODADOS_ENEM_2019.csv')

Recebendo os dados do arquivo .csv. Será dividido em partes (*chunks*) visto que o arquivo é grande.

In [4]:
data_path = '/content/DADOS/MICRODADOS_ENEM_2019.csv'
chunk_enem2019 = pd.read_csv(data_path, sep=';', chunksize=1000000, encoding='latin-1')   # a codificacao do arquivo nao e a padrao UTF-8 

Pré-processando cada *chunk* antes de concatenar todos num só dataframe.


In [5]:
def preprocessing_chunk(chunk):

  columns_to_drop = ['CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
                   'CO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'SG_UF_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO',
                   'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC','CO_UF_ESC', 'SG_UF_ESC',
                   'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA', 'CO_PROVA_CN',
                   'TP_LINGUA', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 
                   'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH',
                   'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
                   'TX_GABARITO_MT', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
                   'TP_STATUS_REDACAO']  

  # excluindo os ausentes na prova
  df_presentes = chunk[chunk.TP_PRESENCA_CN==1]
  df_presentes = df_presentes[df_presentes.TP_PRESENCA_CH==1]
  df_presentes = df_presentes[df_presentes.TP_PRESENCA_LC==1]
  df_presentes = df_presentes[df_presentes.TP_PRESENCA_MT==1]
  df_presentes = df_presentes[df_presentes.TP_STATUS_REDACAO==1]
  
  # excluindo colunas desnecessarias
  df = df_presentes.drop(labels=columns_to_drop, axis=1)

  print(df.shape)
  return df

Concatenando as partes pre-processadas

In [6]:
%%time
list_enem2019 = []

for chunk in chunk_enem2019:
  chunk_preprocessed = preprocessing_chunk(chunk)
  list_enem2019.append(chunk_preprocessed)
df_enem2019 = pd.concat(list_enem2019)

(745356, 97)
(728492, 97)
(720233, 97)
(716297, 97)
(628675, 97)
(57064, 97)
CPU times: user 1min 30s, sys: 8.86 s, total: 1min 39s
Wall time: 1min 39s


In [7]:
df_enem2019.shape

(3596117, 97)

In [8]:
%%time
## change data types (to save up memory usage)    
# Economiza em torno de 1 GB RAM - quase metade
list_col_dtype = [col for col in df_enem2019.columns]

for col in df_enem2019.columns:
   if df_enem2019[col].dtypes == np.int64 and col != 'NU_INSCRICAO':
     df_enem2019[col] = df_enem2019[col].astype(np.int32)
   elif df_enem2019[col].dtypes == np.float64:
     df_enem2019[col] = df_enem2019[col].astype(np.float32)

CPU times: user 9.34 s, sys: 1.82 s, total: 11.2 s
Wall time: 11.2 s


# Exploratory Data Analysis

## Configurando coluna de notas

Criando uma coluna com a media das notas das 4 provas e redação

In [9]:
df_enem2019['NU_MEDIA_NOTAS'] = df_enem2019[['NU_NOTA_CN', 'NU_NOTA_CH', 
                                             'NU_NOTA_LC', 'NU_NOTA_MT',
                                             'NU_NOTA_REDACAO']].mean(axis=1, skipna=False)

Apagando as colunas das notas para economizar espaço no disco

In [10]:
df_enem2019 = df_enem2019.drop(labels=['NU_NOTA_CN', 'NU_NOTA_CH',
                                       'NU_NOTA_LC', 'NU_NOTA_MT',
                                       'NU_NOTA_REDACAO'], axis=1)

In [11]:
df_enem2019['NU_MEDIA_NOTAS'].isna().sum()

0

In [12]:
# Excluir caso haja alguma média NaN
# df_enem2019.dropna(subset=['NU_MEDIA_NOTAS'], inplace=True)

## Agrupando e Codificando Dados 

Indicador de pedido de atendimento especializado

In [13]:
list_especial = ['IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA',
                 'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
                 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_AUTISMO',
                 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF']

df_enem2019['IN_ATEND_ESPECIALIZADO'] = df_enem2019.loc[:, list_especial].max(1)

Indicador de pedido de atendimento especifico

In [14]:
list_especifico = ['IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR']

df_enem2019['IN_ATEND_ESPECIFICO'] = df_enem2019.loc[:, list_especifico].max(1)

Indicador de pedidos de recursos especializados e especificos para a realizacao das provas

In [15]:
list_recurso = ['IN_BRAILLE', 'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO',
                'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_TEMPO_ADICIONAL', 'IN_LEITURA_LABIAL',
                'IN_MESA_CADEIRA_RODAS', 'IN_MESA_CADEIRA_SEPARADA', 'IN_APOIO_PERNA',
                'IN_GUIA_INTERPRETE', 'IN_COMPUTADOR', 'IN_CADEIRA_ESPECIAL',
                'IN_CADEIRA_CANHOTO', 'IN_CADEIRA_ACOLCHOADA', 'IN_PROVA_DEITADO',
                'IN_MOBILIARIO_OBESO', 'IN_LAMINA_OVERLAY', 'IN_PROTETOR_AURICULAR',
                'IN_MEDIDOR_GLICOSE', 'IN_MAQUINA_BRAILE', 'IN_SOROBAN', 'IN_MARCA_PASSO',
                'IN_SONDA', 'IN_MEDICAMENTOS', 'IN_SALA_INDIVIDUAL', 'IN_SALA_ESPECIAL',
                'IN_SALA_ACOMPANHANTE', 'IN_MOBILIARIO_ESPECIFICO', 'IN_MATERIAL_ESPECIFICO',
                'IN_NOME_SOCIAL']

# Como ja existe uma coluna 'IN_SEM_RECURSO' que indica se o inscrito
# fez ou nao um pedido de recurso especial ou especifico, basta apagar
# as outras colunas
df_enem2019.drop(labels=list_recurso, axis =1, inplace=True)

In [16]:
df_enem2019.drop(labels=list_especial, axis=1, inplace=True)
df_enem2019.drop(labels=list_especifico, axis=1, inplace=True)

In [17]:
df_enem2019.shape   # Reduziu drasticamente as dimensoes

(3596117, 45)

Vendo a proporção de dados não respondidos ou não informados

In [18]:
df_enem2019.isna().sum()

NU_INSCRICAO                    0
NU_ANO                          0
CO_UF_RESIDENCIA                0
NU_IDADE                        0
TP_SEXO                         0
TP_ESTADO_CIVIL                 0
TP_COR_RACA                     0
TP_NACIONALIDADE                0
TP_ST_CONCLUSAO                 0
TP_ANO_CONCLUIU                 0
TP_ESCOLA                       0
TP_ENSINO                 1531206
IN_TREINEIRO                    0
TP_DEPENDENCIA_ADM_ESC    2674795
TP_LOCALIZACAO_ESC        2674795
TP_SIT_FUNC_ESC           2674795
IN_SEM_RECURSO                  0
Q001                            0
Q002                            0
Q003                            0
Q004                            0
Q005                            0
Q006                            0
Q007                            0
Q008                            0
Q009                            0
Q010                            0
Q011                            0
Q012                            0
Q013          

In [19]:
print('TP_ESCOLA NR: {:.1f}%'.format(df_enem2019[df_enem2019['TP_ESCOLA']==1].shape[0]/df_enem2019.shape[0]*100))
print('TP_ENSINO NaN: {:.1f}%'.format(df_enem2019['TP_ENSINO'].isna().sum()/df_enem2019.shape[0]*100))
print('TP_DEPENDENCIA_ADM_ESC NaN: {:.1f}%'.format(df_enem2019['TP_DEPENDENCIA_ADM_ESC'].isna().sum()/df_enem2019.shape[0]*100))
print('TP_LOCALIZACAO_ESC NaN: {:.1f}%'.format(df_enem2019['TP_LOCALIZACAO_ESC'].isna().sum()/df_enem2019.shape[0]*100))
print('TP_SIT_FUNC_ESC NaN: {:.1f}%'.format(df_enem2019['TP_SIT_FUNC_ESC'].isna().sum()/df_enem2019.shape[0]*100))

TP_ESCOLA NR: 67.3%
TP_ENSINO NaN: 42.6%
TP_DEPENDENCIA_ADM_ESC NaN: 74.4%
TP_LOCALIZACAO_ESC NaN: 74.4%
TP_SIT_FUNC_ESC NaN: 74.4%


Decidiu-se considerar apenas aqueles que responderam na coluna TP_ESCOLA, assumir os dados NaN da coluna TP_ENSINO como 1 (Ensino Regular) e desconsiderar as colunas TP_DEPENDENCIA_ADM_ESC, TP_LOCALIZACAO_ESC e TP_SIT_FUNC_ESC da análise

In [20]:
df_enem2019 = df_enem2019[df_enem2019.TP_ESCOLA!=1]
df_enem2019['TP_ENSINO'].fillna(value=1, axis=0, inplace=True)

columns_to_drop = ['TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']
df_enem2019.drop(labels=columns_to_drop, axis=1, inplace=True)

df_enem2019.reset_index(drop=True, inplace=True)

In [21]:
df_enem2019.TP_ENSINO = df_enem2019.TP_ENSINO.astype('int32')
df_enem2019.head()

,NU_INSCRICAO,NU_ANO,CO_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,IN_SEM_RECURSO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,NU_MEDIA_NOTAS,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,190001004629,2019,15,18,F,1,1,1,2,0,3,1,0,0,H,E,A,D,3,C,D,E,E,C,A,B,C,B,A,B,A,A,D,B,B,D,A,C,B,581.400024,0,0
1,190001004634,2019,15,26,F,0,3,1,2,0,2,1,0,0,B,B,B,B,4,B,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,423.460022,0,0
2,190001004642,2019,15,17,F,1,3,1,2,0,2,1,0,0,B,D,B,B,3,C,A,B,C,A,B,B,A,A,A,A,A,A,B,B,A,D,A,A,A,461.259949,0,0
3,190001004643,2019,15,17,F,1,3,1,2,0,2,1,0,0,E,E,B,B,9,C,A,B,D,A,A,B,A,A,A,A,A,A,D,A,A,C,A,A,A,468.200012,0,0
4,190001004645,2019,15,18,F,1,3,1,2,0,2,1,0,0,E,E,B,B,8,D,A,B,B,A,B,B,A,B,A,A,A,A,B,A,A,B,A,A,A,514.400024,0,0


Organizando as colunas dicotômicas

In [22]:
# Codificando a variavel dicotomica em 0 e 1
df_enem2019.TP_SEXO.replace(to_replace=['M', 'F'], value=[0, 1], inplace=True)
df_enem2019.TP_SEXO = df_enem2019.TP_SEXO.astype('int32')

Organizando as colunas categóricas

In [25]:
list_to_replace = list(string.ascii_uppercase[0:17])
list_value = list(range(0,17))

In [26]:
# Codificando as variáveis categóricas (A, B, C,...) em 1, 2, 3... 
%%time
df_enem2019.replace(to_replace=list_to_replace, value=list_value, inplace=True) # Toma tempo

In [27]:
df_enem2019.head()

,NU_INSCRICAO,NU_ANO,CO_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,IN_SEM_RECURSO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,NU_MEDIA_NOTAS,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,190001004629,2019,15,18,1,1,1,1,2,0,3,1,0,0,7,4,0,3,3,2,3,4,4,2,0,1,2,1,0,1,0,0,3,1,1,3,0,2,1,581.400024,0,0
1,190001004634,2019,15,26,1,0,3,1,2,0,2,1,0,0,1,1,1,1,4,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,423.460022,0,0
2,190001004642,2019,15,17,1,1,3,1,2,0,2,1,0,0,1,3,1,1,3,2,0,1,2,0,1,1,0,0,0,0,0,0,1,1,0,3,0,0,0,461.259949,0,0
3,190001004643,2019,15,17,1,1,3,1,2,0,2,1,0,0,4,4,1,1,9,2,0,1,3,0,0,1,0,0,0,0,0,0,3,0,0,2,0,0,0,468.200012,0,0
4,190001004645,2019,15,18,1,1,3,1,2,0,2,1,0,0,4,4,1,1,8,3,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,514.400024,0,0


In [28]:
df_enem2019.isna().sum()

NU_INSCRICAO              0
NU_ANO                    0
CO_UF_RESIDENCIA          0
NU_IDADE                  0
TP_SEXO                   0
TP_ESTADO_CIVIL           0
TP_COR_RACA               0
TP_NACIONALIDADE          0
TP_ST_CONCLUSAO           0
TP_ANO_CONCLUIU           0
TP_ESCOLA                 0
TP_ENSINO                 0
IN_TREINEIRO              0
IN_SEM_RECURSO            0
Q001                      0
Q002                      0
Q003                      0
Q004                      0
Q005                      0
Q006                      0
Q007                      0
Q008                      0
Q009                      0
Q010                      0
Q011                      0
Q012                      0
Q013                      0
Q014                      0
Q015                      0
Q016                      0
Q017                      0
Q018                      0
Q019                      0
Q020                      0
Q021                      0
Q022                

Visualização da comparação das variáveis *TP_SEXO* e *IN_ATEND_ESPECIFICO* (que apresentaram correlações maiores) com efeito na Nota Média no ENEM 2019 e da distribuição das notas entre as categorias.

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(28, 17))

# TP_SEXO - BOXPLOT
g1 = sns.boxplot(data=df_enem2019,
                 x='TP_SEXO',
                 y='NU_MEDIA_NOTAS',
                 ax=ax[0][0])
g1.set_title('Representação em Quartiles da variável TP_SEXO')
g1.set_xticklabels(['Masculino', 'Feminino'])     # Sendo 0 - Masc e 1 - Fem
g1.set_xlabel('Sexo')
g1.set_ylabel('Nota Média')

# TP_SEXO - KDE
g2 = sns.kdeplot(data=df_enem2019,
                 x='NU_MEDIA_NOTAS',
                 hue='TP_SEXO',
                 fill=True,
                 ax=ax[0][1])
g2.set_title('Distribuição da variável TP_SEXO')
g2.legend(['Feminino', 'Masculino']).set_title('Sexo')
g2.set_xlabel('Nota Média')
g2.set_ylabel('Densidade')

# IN_ATEND_ESPECIFICO - BOXPLOT
g3 = sns.boxplot(data=df_enem2019,
                 x='IN_ATEND_ESPECIFICO',
                 y='NU_MEDIA_NOTAS',
                 ax=ax[1][0])
g3.set_title('Representação em Quartiles da variável IN_ATEND_ESPECIFICO')
g3.set_xticklabels(['Sem Atendimento', 'Com Atendimento'])     # Sendo 0 - Masc e 1 - Fem
g3.set_xlabel('Índice de Solicitação de Atendimento Especifico')
g3.set_ylabel('Nota Média')

# IN_ATEND_ESPECIFICO - KDE
g4 = sns.kdeplot(data=df_enem2019,
                 x='NU_MEDIA_NOTAS',
                 hue='IN_ATEND_ESPECIFICO',
                 fill=True,
                 ax=ax[1][1])
g4.set_title('Distribuição da variável IN_ATEND_ESPECIFICO')
g4.legend(['Sim', 'Não']).set_title('Atendimento Especifico')
g4.set_xlabel('Nota Média')
g4.set_ylabel('Densidade')


plt.show()

Plot da distribuição das notas de acordo com as idades

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(28, 14))

# IN_ATEND_ESPECIFICO - BOXPLOT
g3 = sns.boxplot(data=df_enem2019,
                 x='NU_IDADE',
                 y='NU_MEDIA_NOTAS')
g3.set_title('Representação em Quartiles da variável NU_IDADE')
#g3.set_xticklabels(['Sem Atendimento', 'Com Atendimento'])     # Sendo 0 - Masc e 1 - Fem
g3.set_xlabel('Idades')
g3.set_ylabel('Nota Média')

Plot da distribuição de notas por estados brasileiros

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(28, 14))

# IN_ATEND_ESPECIFICO - BOXPLOT
g3 = sns.boxplot(data=df_enem2019,
                 x='CO_UF_RESIDENCIA',
                 y='NU_MEDIA_NOTAS')
g3.set_title('Representação em Quartiles da variável NU_IDADE')
#g3.set_xticklabels(['Sem Atendimento', 'Com Atendimento'])     # Sendo 0 - Masc e 1 - Fem
g3.set_xlabel('Idades')
g3.set_ylabel('Nota Média')

# Salvando o DataFrame

Salvar o DataFrame em um arquivo .csv para a utilização de algoritmo de ML.

In [ ]:
savedata_path = '/content/drive/MyDrive/Colab Notebooks/Dados/DF_ENEM_2019.csv'
with open(savedata_path, 'w') as f:
  df_enem2019.to_csv(f, index=False, encoding='latin-1')

In [ ]:
df_enem2019.shape